In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
# Read data and concatenate
DIRECTORY = '../data/'
NAMEBASE = 'EPA_OD_'
DATAYEARS = ['2021', '2022']

df_pm25 = pd.DataFrame()
cols = []
for yr in DATAYEARS:
    for month in range(1, 13):
        month = str(month).zfill(2)
        path = DIRECTORY + NAMEBASE + yr + month + '.csv'
        if not os.path.exists(path):
            break
        df = pd.read_csv(path, low_memory=False)
        if len(cols) == 0:
            cols = df.columns
        assert (cols == df.columns).all(), f'Columns do not match {yr}{month}\n{cols}\n{df.columns}'
        df_pm25 = pd.concat([df_pm25, df], ignore_index=True)
df_pm25.info()

In [ ]:
df_pm25.to_csv(DIRECTORY + 'pm25.csv', index=False)